In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import log_loss

In [ ]:
label_cols = ["Adequate", "Effective", "Ineffective"]

In [ ]:
df = pd.read_csv("../data/train_folded.csv")
oof = pd.read_csv("../data/lgb_162_cv.csv").set_index("discourse_id")
oof = oof.loc[df.discourse_id]
oof = oof[["new_Adequate", "new_Effective", "new_Ineffective"]].values

y = np.zeros((len(df), 3))

for ii, jj in enumerate([label_cols.index(x) for x in df["discourse_effectiveness"].values]):
    y[ii,jj] = 1
    
log_loss(y, oof)

In [ ]:
for i, l in enumerate(label_cols):
    df["discourse_effectiveness_"+l] = oof[:,i]

In [ ]:
folds = [
    "ff"
]

for FOLD in folds:
    df = pd.read_csv("../data/train_folded.csv")
    df_old = pd.read_csv("../data/old_competition_data.csv")
    df_old["discourse_type_essay"] = df_old.groupby("essay_id")["discourse_type"].transform(lambda x: " ".join(x)).values
    ps = []
    for seed in range(1,2):
        if FOLD == "ff":
            df_pseudo = pd.read_csv(f"../data/pseudo_162_ff_raw.csv")
        ps.append(df_pseudo[label_cols])

    p = np.mean(ps, axis=0)

    df_pseudo[label_cols] = p

    df_pseudo["fold"] = -1
    
    print(p.mean(axis=0))
    
    df_pseudo = df_pseudo.rename(columns={k: "discourse_effectiveness_"+k for k in label_cols})
        
    df_pseudo = df_old.merge(df_pseudo, left_on="discourse_id", right_on="discourse_id", how="left")
    
    if FOLD != "ff":
        df = df[df.fold==FOLD]
    df_pseudo = pd.concat([df_pseudo, df], axis=0)
    
    df_pseudo.to_csv(f"../data/pseudoold_oof_164_fold_{FOLD}.csv", index=False)